In [1]:
import io
import json
import zipfile

import httpx
from rich import print as rprint
from tqdm import tqdm

# API

In [2]:
class CamaraAPI:
    def __init__(self) -> None:
        self.base_url = "https://dadosabertos.camara.leg.br/api/v2"

    def request(self, endpoint: str) -> dict:
        response = httpx.get(f"{self.base_url}/{endpoint}")
        return response.json()

    def get_deputados(self) -> dict:
        return self.request("deputados").get("dados", {})

    def get_despesas(self, id_: int, year: int = 2022) -> dict:
        return self.request(f"deputados/{id_}/despesas?ano={year}")

In [3]:
despesas = []

api = CamaraAPI()

deputados = api.get_deputados()
for deputado in tqdm(deputados):
    id_ = deputado["id"]
    despesas_deputado = api.get_despesas(id_=id_, year=2022)
    despesas.extend(despesas_deputado["dados"])
rprint(len(despesas))

100%|██████████| 512/512 [01:37<00:00,  5.25it/s]


31

In [4]:
rprint(deputados[0])

{
    'id': 204379,
    'uri': 'https://dadosabertos.camara.leg.br/api/v2/deputados/204379',
    'nome': 'Acácio Favacho',
    'siglaPartido': 'MDB',
    'uriPartido': 'https://dadosabertos.camara.leg.br/api/v2/partidos/36899',
    'siglaUf': 'AP',
    'idLegislatura': 57,
    'urlFoto': 'https://www.camara.leg.br/internet/deputado/bandep/204379.jpg',
    'email': 'dep.acaciofavacho@camara.leg.br'
}

In [5]:
rprint(despesas[0])

{
    'ano': 2022,
    'mes': 12,
    'tipoDespesa': 'LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES',
    'codDocumento': 7499778,
    'tipoDocumento': 'Nota Fiscal',
    'codTipoDocumento': 0,
    'dataDocumento': '2023-02-17T00:00:00',
    'numDocumento': '635',
    'valorDocumento': 3090.0,
    'urlDocumento': 'https://www.camara.leg.br/cota-parlamentar/documentos/publ/1758/2023/7499778.pdf',
    'nomeFornecedor': 'LIZIANE QUEVEDO',
    'cnpjCpfFornecedor': '07086926000190',
    'valorLiquido': 3090.0,
    'valorGlosa': 0.0,
    'numRessarcimento': '',
    'codLote': 1909470,
    'parcela': 0
}

# URL

In [6]:
def get_despesas(url: str) -> dict[str, list[dict[str, list]]]:
    response = httpx.get(url=url)
    zip_content = response.content

    with zipfile.ZipFile(io.BytesIO(zip_content)) as zip_file:
        with zip_file.open(zip_file.namelist()[0]) as json_file:
            json_content = json_file.read().decode("utf-8")

    return json.loads(json_content).get("dados")


url = "https://www.camara.leg.br/cotas/Ano-2022.json.zip"
despesas = get_despesas(url=url)
rprint(len(despesas))

209227

In [7]:
rprint(despesas[0])

{
    'nomeParlamentar': 'LIDERANÇA DO CIDADANIA',
    'cpf': '',
    'numeroCarteiraParlamentar': '',
    'legislatura': 2019,
    'siglaUF': 'NA',
    'siglaPartido': '',
    'codigoLegislatura': 56,
    'numeroSubCota': 4,
    'descricao': 'CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.',
    'numeroEspecificacaoSubCota': 0,
    'descricaoEspecificacao': '',
    'fornecedor': 'KAWAMURA, CALHADO E VILELA SOCIEDADE DE ADVOGADOS',
    'cnpjCPF': '166.923.980/0010-4 ',
    'numero': '1',
    'tipoDocumento': '0',
    'dataEmissao': '2023-02-15T00:00:00',
    'valorDocumento': '28500',
    'valorGlosa': '0',
    'valorLiquido': '28500',
    'mes': 12,
    'ano': 2022,
    'parcela': 0,
    'passageiro': '',
    'trecho': '',
    'lote': '1899877',
    'ressarcimento': '',
    'datPagamentoRestituicao': '',
    'restituicao': '',
    'numeroDeputadoID': 3453,
    'idDocumento': 7481981,
    'urlDocumento': 'https://www.camara.leg.br/cota-parlamentar/documentos/publ/3453/2022/7481981.pdf'
}